In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
from fastai import *
from fastai.tabular import * 
from fastai.text import *
from exp.eventlog import *

import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

In [3]:
path = untar_data(URLs.BPIC_2012)
log = import_xes(path,extensions=False,classifiers=False,schema=False,log_attributes=False)

failed to parse date: 1970-01-01T00:00:00.000+01:00
failed to parse date: 1970-01-01T00:00:00.000+01:00
failed to parse date: 2012-04-23T00:00:00.000+02:00
failed to parse date: 2011-10-01T00:38:44.546+02:00
failed to parse date: 2012-03-14T16:04:54.681+01:00


In [4]:
df = log.events
df.head()

,trace_id,event_id,org:resource,lifecycle:transition,concept:name,time:timestamp
0,173688,0,112,COMPLETE,A_SUBMITTED,2011-09-30 22:38:44.546000+00:00
1,173688,1,112,COMPLETE,A_PARTLYSUBMITTED,2011-09-30 22:38:44.880000+00:00
2,173688,2,112,COMPLETE,A_PREACCEPTED,2011-09-30 22:39:37.906000+00:00
3,173688,3,112,SCHEDULE,W_Completeren aanvraag,2011-09-30 22:39:38.875000+00:00
4,173688,4,None,START,W_Completeren aanvraag,2011-10-01 09:36:46.437000+00:00


# Preprocessing

### Integer-encoding the unique values of the desired columns

In [5]:
activities = df['concept:name'].unique()
people = df['org:resource'].unique()

concepts = tuple(set(activities))
int2concept = dict(enumerate(concepts))
concept2int = {c: ii for ii, c in int2concept.items()}

concepts_enc = np.array([concept2int[c] for c in activities])

users = tuple(set(people))
int2user = dict(enumerate(users))
user2int = {u: ii for ii, u in int2user.items()}

users_enc = np.array([user2int[c] for c in people])

In [6]:
print("There are " + str(len(activities)) + " unique values in concept:names.")
print("There are " + str(len(people)) + " unique values in org:resources.")

There are 24 unique values in concept:names.
There are 69 unique values in org:resources.


In [7]:
display(concepts_enc)
display(users_enc)

array([20,  3, 14,  5, 23, 11, 16, 12,  1, 10,  9, 15, 17,  4,  7, 21, 19,  2, 13,  8,  6, 18, 22,  0])

array([53, 31, 46, 58, ...,  6, 49, 27, 33])

In [8]:
pd.Categorical??

### Encoding the desired column in the dataframe

Hier fehlt allerdings leider die Möglichkeit, das ganze wieder zurück encoden zu können. Ich weiss nicht wie man aus den Integern hier wieder die Strings bekommt, oder wo man das dazu gehörige mapping findet.

In [9]:
labels, levels = pd.factorize(df["concept:name"])

df["concept:name"] = df["concept:name"].astype('category')
df["concept:name"] = df["concept:name"].cat.codes

df.head()

,trace_id,event_id,org:resource,lifecycle:transition,concept:name,time:timestamp
0,173688,0,112,COMPLETE,9,2011-09-30 22:38:44.546000+00:00
1,173688,1,112,COMPLETE,6,2011-09-30 22:38:44.880000+00:00
2,173688,2,112,COMPLETE,7,2011-09-30 22:39:37.906000+00:00
3,173688,3,112,SCHEDULE,19,2011-09-30 22:39:38.875000+00:00
4,173688,4,None,START,19,2011-10-01 09:36:46.437000+00:00


In [10]:
CategoryProcessor??

### Traces zusammenfassen

So, dass alles über einen Trace in einer Zeile steht mit der trace_id als Index.

In [11]:
# convert traces generalized
def conv_traces(df):
    d = {}
    for i in df.columns:
        d[i] = ' '.join(df[i].map(str))
        
    return Series(d)

In [12]:
df = df.groupby('trace_id')["concept:name", "org:resource"].apply(conv_traces)
df.head()

,concept:name,org:resource
trace_id,,
173688,9 6 7 19 19 0 14 5 12 15 21 19 21 21 21 21 21 ...,112 112 112 112 None 10862 10862 10862 10862 1...
173691,9 6 7 19 19 19 19 0 5 14 12 15 21 19 21 21 21 ...,112 112 112 112 None None None 10862 10862 108...
173694,9 6 7 19 19 19 19 19 19 19 19 0 5 14 12 15 21 ...,112 112 112 112 10912 10912 None None 11201 11...
173697,9 6 4,112 112 112
173700,9 6 4,112 112 112


Nachfolgend werden alle concept:name Werte die aktuell noch in Strings zusammengefasst sind in eine große Liste eingefügt. So bekommt man einen großen Datensatz (/Liste) aller Werte in der Spalte. 

Hier wäre es sinnvoll am Ende jedes Traces irgendein bestimmtes encodetes Token einzufügen, oder nicht?!

In [13]:
%%time

data = np.array([])

for i in range(len(df)):
    data = np.append(data, [df["concept:name"][i].split()])

CPU times: user 49.7 s, sys: 0 ns, total: 49.7 s
Wall time: 49.7 s


In [14]:
def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array
    one_hot = np.zeros((np.multiply(*arr.shape), n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [15]:
def get_batches(arr, n_seqs, n_steps):
    '''Create a generator that returns batches of size
       n_seqs * n_steps from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       n_seqs: Batch size, the number of sequences per batch
       n_steps: Number of sequence steps per batch
    '''
    
    batch_size = n_seqs * n_steps
    n_batches = len(arr)//batch_size
    
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size]
    
    # Reshape into n_seqs rows
    arr = arr.reshape((n_seqs, -1))
    
    for n in range(0, arr.shape[1], n_steps):
        
        # The features
        x = arr[:, n:n+n_steps]
        
        # The targets, shifted by one
        y = np.zeros_like(x)
        
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+n_steps]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

In [16]:
batches = get_batches(data, 10, 20)
x, y = next(batches)

In [17]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

display(x, y)

array([['9', '6', '7', '19', ..., '21', '16', '22', '21'],
       ['21', '16', '22', '21', ..., '9', '6', '7', '19'],
       ['19', '19', '19', '19', ..., '21', '11', '14', '12'],
       ['21', '21', '21', '16', ..., '22', '10', '8', '2'],
       ...,
       ['17', '17', '4', '17', ..., '5', '14', '12', '15'],
       ['20', '22', '20', '20', ..., '20', '20', '20', '20'],
       ['16', '22', '21', '22', ..., '19', '0', '14', '5'],
       ['7', '19', '19', '0', ..., '21', '21', '21', '21']], dtype='<U32')

array([['6', '7', '19', '19', ..., '16', '22', '21', '22'],
       ['16', '22', '21', '22', ..., '6', '7', '19', '19'],
       ['19', '19', '19', '19', ..., '11', '14', '12', '15'],
       ['21', '21', '16', '22', ..., '10', '8', '2', '1'],
       ...,
       ['17', '4', '17', '9', ..., '14', '12', '15', '21'],
       ['22', '20', '20', '20', ..., '20', '20', '20', '20'],
       ['22', '21', '22', '10', ..., '0', '14', '5', '12'],
       ['19', '19', '0', '14', ..., '21', '21', '21', '11']], dtype='<U32')

# Defining the LSTM Model

In [18]:
class LSTM(nn.Module):
    
    def __init__(self, tokens, n_steps=100, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # Creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        ## Define the LSTM
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        ## Define a dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        ## Define the final, fully-connected output layer
        self.fc = nn.Linear(n_hidden, len(self.chars))
        
        # Initialize the weights
        self.init_weights()
      
    
    def forward(self, x, hc):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hc`. '''
        
        ## Get x, and the new hidden state (h, c) from the lstm
        x, (h, c) = self.lstm(x, hc)
        
        ## Ppass x through the dropout layer
        x = self.dropout(x)
        
        # Stack up LSTM outputs using view
        x = x.view(x.size()[0]*x.size()[1], self.n_hidden)
        
        ## Put x through the fully-connected layer
        x = self.fc(x)
        
        # Return x and the hidden state (h, c)
        return x, (h, c)
    
    
    def predict(self, char, h=None, cuda=False, top_k=None):
        ''' Given a character, predict the next character.
        
            Returns the predicted character and the hidden state.
        '''
        if cuda:
            self.cuda()
        else:
            self.cpu()
        
        if h is None:
            h = self.init_hidden(1)
        
        x = np.array([[self.char2int[char]]])
        x = one_hot_encode(x, len(self.chars))
        
        inputs = torch.from_numpy(x)
        
        if cuda:
            inputs = inputs.cuda()
        
        h = tuple([each.data for each in h])
        out, h = self.forward(inputs, h)

        p = F.softmax(out, dim=1).data
        
        if cuda:
            p = p.cpu()
        
        if top_k is None:
            top_ch = np.arange(len(self.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        p = p.numpy().squeeze()
        
        char = np.random.choice(top_ch, p=p/p.sum())
            
        return self.int2char[char], h
    
    def init_weights(self):
        ''' Initialize weights for fully connected layer '''
        initrange = 0.1
        
        # Set bias tensor to all zeros
        self.fc.bias.data.fill_(0)
        # FC weights as random uniform
        self.fc.weight.data.uniform_(-1, 1)
        
    def init_hidden(self, n_seqs):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x n_seqs x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        return (weight.new(self.n_layers, n_seqs, self.n_hidden).zero_(),
                weight.new(self.n_layers, n_seqs, self.n_hidden).zero_())

## Encoding by FastAI